Hi guys, from past few days I am learning about Time series analysis and forcasting, So in this kernel going to share what I have learned so far. In layman terms, I can simply define it as 'time based analysis of any fact' and is required when target is mostly dependent on date & time variable.

If you are a begineer then I would suggest below resources to go through, they helped me alot.

https://www.youtube.com/watch?v=Aw77aMLj9uM

https://www.analyticsvidhya.com/blog/2016/02/time-series-forecasting-codes-python/

Below are the major steps that I am going to perform in this kernel : 
* **Import required libraries**
* **Feature Engineering**
* **Data Cleaning**
* **Exploratory Data Analysis & Visualizations**
* **Time series Prediction**

## <a>Import required libraries</a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import seaborn as sns # for plot visualization
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller, acf, pacf

import os
print(os.listdir("../input"))

There is column named **datetime_utc** in this dataset, we are going to read that as an index.

In [ ]:
weather_df = pd.read_csv('../input/testset.csv', parse_dates=['datetime_utc'], index_col='datetime_utc')
weather_df.head()

## <a>Feature Engineering</a>

Here we are going to consider only few of the columns which seems important from some basic EDA and time series prediction's point of view. At the same time renaming with some better ones.

In [ ]:
weather_df = weather_df.loc[:,[' _conds', ' _hum', ' _tempm']]
weather_df = weather_df.rename(index=str, columns={' _conds': 'condition', ' _hum': 'humidity', ' _pressurem': 'pressure', ' _tempm': 'temprature'})
print(f'dataset shape (rows, columns) - {weather_df.shape}')
weather_df.head()

In [ ]:
# lets check dtype of all columns, 
weather_df.dtypes, weather_df.index.dtype

It shows 'index' as object type which needs to be converted to datetime otherwise we won't be able to perform scaling during time series analysis.

In [ ]:
weather_df.index = pd.to_datetime(weather_df.index)
weather_df.index

Seems fine. Now lets check the total number (and percent) of missing values in each columns.

## <a>Data Cleaning</a>

In [ ]:
def list_and_visualize_missing_data(dataset):
    # Listing total null items and its percent with respect to all nulls
    total = dataset.isnull().sum().sort_values(ascending=False)
    percent = ((dataset.isnull().sum())/(dataset.isnull().count())).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    missing_data = missing_data[missing_data.Total > 0]
    
    missing_data.plot.bar(subplots=True, figsize=(16,9))

list_and_visualize_missing_data(weather_df)

Not many values are missing, but it will still be great to fill the missing ones instead of removing entire row.

In [ ]:
# will fill with previous valid value
weather_df.ffill(inplace=True)
weather_df[weather_df.isnull()].count()

In [ ]:
weather_df.describe()

It is showing maximum temprature as 90 and max humidity as 243 which is non-realistic, so is an outlier. We need to treat these outliers.

In [ ]:
weather_df = weather_df[weather_df.temprature < 50]
weather_df = weather_df[weather_df.humidity <= 100]

## <a>Exploratory Data Analysis & Visualizations</a>

Let's checkout the most common weather condition type in Delhi.

In [ ]:
weather_condition = (weather_df.condition.value_counts()/(weather_df.condition.value_counts().sum()))*100
weather_condition.plot.bar(figsize=(16,9))
plt.xlabel('Weather Conditions')
plt.ylabel('Percent')

Oh, it is really bad that mostly Delhi has Hazy and smoky weather conditions, it shows the level of pollution city has.

Let's see how plot for all year's temprature and humidity looks like.

In [ ]:
weather_df.plot(subplots=True, figsize=(20,12))

It seems overplotted, let's plot for only two years 2015 and 2016, it will give us the clear picture of seasonality and tread.

In [ ]:
weather_df['2015':'2016'].asfreq('D', method='pad').resample('D').mean().plot(subplots=True, figsize=(20,12))

So it shows that mid two quarter are hotter than first and last, humidity varies accordingly.

### Check Stationarity
So above plots shows that we do have seasonality but there is no trend, because of this seasonality it is non-stationary right now. Still let's check for below necessary conditions:

* Constant mean
* Constant variance
* An auto co-variance that does not depend on time

In [ ]:
# check rolling mean and rolling standard deviation
def plot_rolling_mean_std(ts):
    rolling_mean = ts.rolling(12).mean()
    rolling_std = ts.rolling(12).std()
    plt.figure(figsize=(22,10))

    plt.plot(ts, label='Actual Mean')
    plt.plot(rolling_mean, label='Rolling Mean')
    plt.plot(rolling_std, label = 'Rolling Std')
    plt.xlabel("Date")
    plt.ylabel("Mean Temperature")
    plt.title('Rolling Mean & Rolling Standard Deviation')
    plt.legend()
    plt.show()

In [ ]:
# Augmented Dickey–Fuller test
def perform_dickey_fuller_test(ts):
    result = adfuller(ts, autolag='AIC')
    print('Test statistic: ' , result[0])
    print('Critical Values:' ,result[4])

In Dickey-Fuller test, we need only test_statics and critical_value to know if it is stationary or not

In [ ]:
# check stationary: mean, variance(std)and adfuller test
weather_ts_resampled = weather_df['2015':'2016'].temprature.asfreq('D', method='pad').resample('D').mean()
plot_rolling_mean_std(weather_ts_resampled)
perform_dickey_fuller_test(weather_ts_resampled)

After observing, we can clearly say that -
* We don't have constant mean.
* Here we can consider variance as constant.
* As per Dickey-Fuller test result, our test statistic is no where less than Critical values.

So, it can be concluded that it is non-stationary.

### Make Stationary
For non-stationary to stationary conversion, we can use any of the below technique : 

* Decomposing
* Differencing

Here, we are preferring Differencing because it is very straight forward.

In [ ]:
weather_ts_resampled_log = np.log(weather_ts_resampled)
weather_ts_resampled_log_stationary = weather_ts_resampled_log - weather_ts_resampled_log.shift()
# due to shifting there is one NaN value
weather_ts_resampled_log_stationary.dropna(inplace=True)

plot_rolling_mean_std(weather_ts_resampled_log_stationary)
perform_dickey_fuller_test(weather_ts_resampled_log_stationary)

Now it seems somewhat acceptable to me

* We have constant mean.
* Nearly constant variance.
* As per Dickey-Fuller test result, our test statistic is less than Critical values.

## <a>Time series Prediction</a>

For prediction we are going to use one of the most popular model for time series, **Autoregressive Integrated Moving Average (ARIMA)** which is a standard statistical model for time series forecast and analysis.
An ARIMA model can be understood by outlining each of its components as follows:
* **Autoregression (AR) -** refers to a model that shows a changing variable that regresses on its own lagged, or prior, values.<br/>
The notation **AR(p)** indicates an autoregressive model of order p.

    *Example* — If p is 3 the predictor for X(t) will be 
        X(t) = µ + X(t-1) + X(t-2) + X(t-3) + εt

    Where ε is error term.
* **Integrated (I) -** represents the differencing of raw observations to allow for the time series to become stationary, i.e., data values are replaced by the difference between the data values and the previous values.
* **Moving average (MA) -** incorporates the dependency between an observation and a residual error from a moving average model applied to lagged observations.

    The notation **MA(q)** refers to the moving average model of order q:<br/>


    *Example* — If q is 3 the predictor for X(t) will be 
        X(t) = µ + εt + θ1.ε(t-1) + θ2.ε(t-2) + θ3.ε(t-3)
    Here instead of difference from previous term, we take errer term (ε) obtained from the difference from past term
Now we need to figure out the values of p and q which are parameters of ARIMA model. We use below two methods to figure out these values  -

**Autocorrelation Function (ACF):** It just measures the correlation between two consecutive (lagged version). example at lag 4, ACF will compare series at time instance t1…t2 with series at instance t1–4…t2–4

**Partial Autocorrelation Function (PACF):** is used to measure the degree of association between X(t) and X(t-p).

In [ ]:
acf_lag = acf(weather_ts_resampled_log_stationary, nlags=20)
pacf_lag = pacf(weather_ts_resampled_log_stationary, nlags=20, method='ols')

plt.figure(figsize=(22,10))

plt.subplot(121)
plt.plot(acf_lag)
plt.axhline(y=0,linestyle='--',color='silver')
plt.axhline(y=-1.96/np.sqrt(len(weather_ts_resampled_log_stationary)),linestyle='--',color='silver')
plt.axhline(y=1.96/np.sqrt(len(weather_ts_resampled_log_stationary)),linestyle='--',color='silver')
plt.title("Autocorrelation Function")

plt.subplot(122)
plt.plot(pacf_lag)
plt.axhline(y=0,linestyle='--',color='silver')
plt.axhline(y=-1.96/np.sqrt(len(weather_ts_resampled_log_stationary)),linestyle='--',color='silver')
plt.axhline(y=1.96/np.sqrt(len(weather_ts_resampled_log_stationary)),linestyle='--',color='silver')
plt.title("Partial Autocorrelation Function")
plt.tight_layout()

These grey dotted line are confidence intervels which we are going to use to find out the value of p and q.

* **p - **the point where PACF crosses the upper confiednce level. In our case it seems to be 1. So we will take **p = 1**.
* **q -** the point where ACF crosses the upper confiednce level. In our case it seems to be 1. So we will take **q = 1**.
* **d -** number of nonseasonal differences needed for stationarity. In this case we are going to take it as 0, since we already have stationary time series.

Now we are going fit time series for AR, MA & ARIMA Models. We will compare performance on the basis of RSS score and at last prefer the best one.

### 1. AR Model

In [ ]:
model = ARIMA(weather_ts_resampled_log_stationary, order=(1,0,0))
AR_model = model.fit(disp=1)
print(f'AR Model RSS Score - {sum((AR_model.fittedvalues - weather_ts_resampled_log_stationary)**2)}')
print(f'AR Model RMSE - {np.sqrt(sum((AR_model.fittedvalues-weather_ts_resampled_log_stationary)**2)/len(weather_ts_resampled_log_stationary))}')
plt.figure(figsize=(22,10))
plt.plot(weather_ts_resampled_log_stationary, label='Actual')
plt.plot(AR_model.fittedvalues, label='Predicted')
plt.title('AR Model')
plt.legend()
plt.show()

### 2. MA Model

In [ ]:
model = ARIMA(weather_ts_resampled_log_stationary, order=(0,0,1))
MA_model = model.fit(disp=1)
print(f'MA Model RSS Score - {sum((MA_model.fittedvalues - weather_ts_resampled_log_stationary)**2)}')
print(f'MA Model RMSE - {np.sqrt(sum((MA_model.fittedvalues-weather_ts_resampled_log_stationary)**2)/len(weather_ts_resampled_log_stationary))}')
plt.figure(figsize=(22,10))
plt.plot(weather_ts_resampled_log_stationary, label='Actual')
plt.plot(MA_model.fittedvalues, label='Predicted')
plt.title('MA Model')
plt.legend()
plt.show()

### 3. ARIMA Model

In [ ]:
model = ARIMA(weather_ts_resampled_log_stationary, order=(1,0,1))
ARIMA_model = model.fit(disp=1)
print(f'ARIMA Model RSS Score - {sum((ARIMA_model.fittedvalues - weather_ts_resampled_log_stationary)**2)}')
print(f'ARIMA Model RMSE - {np.sqrt(sum((ARIMA_model.fittedvalues-weather_ts_resampled_log_stationary)**2)/len(weather_ts_resampled_log_stationary))}')
plt.figure(figsize=(22,10))
plt.plot(weather_ts_resampled_log_stationary, label='Actual')
plt.plot(ARIMA_model.fittedvalues, label='Predicted')
plt.title('ARIMA Model')
plt.legend()
plt.show()

### Bringing this back to original scale

In [ ]:
ARIMA_predictions = pd.Series(ARIMA_model.fittedvalues, copy=True)
ARIMA_predictions.head()

In [ ]:
def inverse_difference(last_ob, value):
    return value + last_ob

ARIMA_predictions_inverse_difference = pd.Series(index=ARIMA_predictions.index, data=[inverse_difference(weather_ts_resampled_log[key], ARIMA_predictions[key]) for key,value in enumerate(ARIMA_predictions)])
ARIMA_predictions_inverse_difference.head()

As expected first value is missing, which was a windows size, and we removed it durigin shift (while converting to stationary) because of NaN. 
We have two option to handle this case - 
1. We create this key and put the value from actual record.
2. We don't consider this key while comparing actual & prediction and caluting RMSE.

I am going to follow the second appraoch.

In [ ]:
missing_keys = weather_ts_resampled_log.index.difference(ARIMA_predictions_inverse_difference.index)
missing_keys.values

In [ ]:
final_ARIMA_predictions = np.exp(ARIMA_predictions_inverse_difference)
plt.figure(figsize=(22,10))
actual_values = weather_ts_resampled.drop(missing_keys.values)
plt.plot(actual_values, label='Actual')
plt.plot(final_ARIMA_predictions, label='Prediction')

plt.title(f'RMSE - {np.sqrt(sum((final_ARIMA_predictions-actual_values)**2)/len(actual_values))}')
plt.legend()
plt.show()

Looks good. This is it from my side for now, will do some more improvements later. 

Will be glad to have your suggestions. Thanks ;)